In [1]:
import xarray as xr
import plotly.express as px
import plotly.subplots

In [2]:
data = xr.open_dataset("../1_Forcing/bats_dataset.zarr", engine="zarr")
data.load()

<xarray.Dataset> Size: 15MB
Dimensions:                        (time: 4242, depth: 8, latitude: 1,
                                    longitude: 1, layer: 3, sieve_size: 5)
Coordinates:
  * depth                          (depth) float64 64B 50.0 100.0 ... 400.0
  * latitude                       (latitude) float64 8B 31.6
  * layer                          (layer) float32 12B 1.0 2.0 3.0
  * longitude                      (longitude) float64 8B -64.2
  * sieve_size                     (sieve_size) float64 40B 0.2 0.5 1.0 2.0 5.0
  * time                           (time) datetime64[ns] 34kB 1988-10-21T04:5...
Data variables: (12/52)
    CO2                            (time, depth, latitude, longitude) float64 271kB ...
    CTD_salinity                   (time, depth, latitude, longitude) float64 271kB ...
    POC                            (time, depth, latitude, longitude) float64 271kB ...
    PON                            (time, depth, latitude, longitude) float64 271kB ...
    POP                            (time, depth, latitude, longitude) float64 271kB ...
    TN                             (time, depth, latitude, longitude) float64 271kB ...
    ...                             ...
    synechococcus                  (time, depth, latitude, longitude) float64 271kB ...
    t0                             (time, depth, latitude, longitude) float64 271kB ...
    temp                           (time, depth, latitude, longitude) float64 271kB ...
    total_dissolved_phosphorus     (time, depth, latitude, longitude) float64 271kB ...
    wet_weight_migrant             (sieve_size, time, latitude, longitude, depth) float64 1MB ...
    wet_weight_resident            (sieve_size, time, latitude, longitude, depth) float64 1MB ...
Attributes:
    Conventions:       CF-1.11
    history:           Created on 2022-09-28
    institution:       CLS
    references:        http://www.cls.fr; http://www.seapodym.eu
    source:            SEAPODYM-LMTL 3.0.0
    subset:datasetId:  cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_202211
    subset:date:       2024-06-12T03:45:47.386Z
    subset:productId:  GLOBAL_MULTIYEAR_BGC_001_033
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    title:             Global ocean low and mid trophic levels biomass conten...

In [4]:
seapodym_zoo = data.cmems_zooc[:, 0, 0].dropna("time")

obs_zoo = data.dry_weight_resident.sum(["sieve_size"]).mean("depth")
obs_zoo = obs_zoo.where(obs_zoo > 0).dropna("time")[:, 0, 0]

## Comparison


In [5]:
zoo_line = xr.merge([seapodym_zoo, obs_zoo]).to_dataframe().drop(columns=["latitude", "longitude"])

diff_zoo = ((obs_zoo - seapodym_zoo) / obs_zoo) * 100
diff_zoo.name = "diff_zoo"
diff_zoo = diff_zoo.to_dataframe().reset_index().drop(columns=["latitude", "longitude"])
diff_zoo = diff_zoo.resample("3ME", on="time").mean()["diff_zoo"]

In [9]:
fig = plotly.subplots.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0, row_heights=[3, 1])

bats_zoo = zoo_line["cmems_zooc"].dropna()
fig.add_scatter(x=bats_zoo.index, y=bats_zoo, marker={"color": "blue"}, row=1, col=1, name="BATS station")
cmems_zoo = zoo_line["dry_weight_resident"].dropna()
fig.add_scatter(x=cmems_zoo.index, y=cmems_zoo, marker={"color": "red"}, row=1, col=1, name="Seapodym")


fig.add_bar(x=diff_zoo.index, y=diff_zoo, row=2, col=1, marker={"color": diff_zoo}, name="Difference")

fig.update_xaxes({"title": "Time"}, row=2)
fig.update_layout({"title": "Zooplankton comparison between BATS station and Seapodym model"})
fig.update_yaxes({"title": "Zooplankton (g/m2)"}, row=1)
fig.update_yaxes({"title": "Difference in % of BATS zooplankton"}, row=2)
fig.show()

In [10]:
with open("BATS_comparison.html", "w") as f:
    f.write(fig.to_html())